<div dir="rtl" align="right">

# بخش ۸: اجرای موازی ایجنت‌ها در LangChain

در این بخش یاد می‌گیریم چطور چندین ایجنت یا وظیفه را به‌صورت موازی اجرا کنیم تا زمان پاسخ‌دهی را کاهش دهیم و کارایی سیستم را افزایش دهیم.

## اهداف یادگیری
- آشنایی با مفهوم اجرای موازی (Parallel Execution) در LangChain
- یادگیری استفاده از RunnableParallel برای اجرای همزمان چند تسک
- مقایسه زمان اجرای سریالی و موازی
- ساخت یک مثال عملی با چند ایجنت موازی

## فهرست مطالب
1. مقدمه و مفهوم اجرای موازی
2. نصب کتابخانه‌ها و تنظیمات اولیه
3. مثال ساده: اجرای موازی با RunnableParallel
4. مقایسه زمان اجرای سریالی و موازی
5. مثال پیشرفته: ایجنت‌های موازی با ابزارهای مختلف
6. تمرین و جمع‌بندی

</div>

<div dir="rtl" align="right">

---

### ۱. مقدمه و مفهوم اجرای موازی

**چرا اجرای موازی؟**  
زمانی که می‌خواهیم چند وظیفه مستقل را انجام دهیم (مثلاً پرسیدن چند سؤال مختلف از مدل)، اگر به‌صورت سریالی اجرا کنیم، زمان زیادی صرف می‌شود. با اجرای موازی می‌توانیم این وظایف را همزمان انجام دهیم و زمان کل را کاهش دهیم.

**مزایا:**
- کاهش زمان پاسخ‌دهی
- افزایش کارایی سیستم
- امکان مدیریت چند درخواست همزمان

**کاربردها:**
- پردازش همزمان چند سؤال
- تحلیل چند سند به‌صورت موازی
- اجرای ایجنت‌های مستقل با ابزارهای مختلف

</div>

<div dir="rtl" align="right">

---

### ۲. نصب کتابخانه‌ها و تنظیمات اولیه

**کتابخانه‌های مورد نیاز:**
```bash
pip install langchain langchain-openai langchain-groq python-dotenv
```

در این بخش، مدل OpenAI را برای تست‌های موازی راه‌اندازی می‌کنیم.

</div>

In [ ]:
from dotenv import load_dotenv
import os
import time

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.2)

<div dir="rtl" align="right">

---

### ۳. مثال ساده: اجرای موازی با RunnableParallel

**RunnableParallel چیست؟**  
این ابزار به ما اجازه می‌دهد چند زنجیره یا وظیفه را به‌صورت موازی اجرا کنیم. هر کدام از وظایف می‌تواند یک پرامپت جداگانه یا یک chain مستقل باشد.

**مثال:**  
ما سه سؤال مختلف می‌پرسیم و همه را به‌صورت موازی اجرا می‌کنیم.

</div>

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableParallel
from langchain_core.output_parsers import StrOutputParser

# Define three different prompts
prompt1 = PromptTemplate(input_variables=["topic"], template="What is {topic}?")
prompt2 = PromptTemplate(input_variables=["topic"], template="Give me 3 benefits of {topic}.")
prompt3 = PromptTemplate(input_variables=["topic"], template="What are common challenges with {topic}?")

# Create three chains
chain1 = prompt1 | llm | StrOutputParser()
chain2 = prompt2 | llm | StrOutputParser()
chain3 = prompt3 | llm | StrOutputParser()

# Run them in parallel
parallel_chain = RunnableParallel(
    definition=chain1,
    benefits=chain2,
    challenges=chain3
)

result = parallel_chain.invoke({"topic": "machine learning"})

print("Definition:", result["definition"])
print("\nBenefits:", result["benefits"])
print("\nChallenges:", result["challenges"])

<div dir="rtl" align="right">

**توضیح کد:**
1. سه پرامپت مختلف تعریف کردیم که هر کدام سؤال متفاوتی می‌پرسند
2. هر پرامپت را به LLM و یک output parser متصل کردیم
3. با استفاده از `RunnableParallel`، هر سه زنجیره را به‌صورت موازی اجرا می‌کنیم
4. نتایج را به‌صورت یک دیکشنری دریافت می‌کنیم

**نکته:** اجرای موازی زمان کل را نسبت به اجرای سریالی کاهش می‌دهد.

</div>

<div dir="rtl" align="right">

---

### ۴. مقایسه زمان اجرای سریالی و موازی

حالا همان سه سؤال را یک‌بار به‌صورت سریالی و یک‌بار به‌صورت موازی اجرا می‌کنیم و زمان آن‌ها را مقایسه می‌کنیم.

</div>

In [ ]:
# Serial execution
start_serial = time.time()
r1 = chain1.invoke({"topic": "Python programming"})
r2 = chain2.invoke({"topic": "Python programming"})
r3 = chain3.invoke({"topic": "Python programming"})
end_serial = time.time()

serial_time = end_serial - start_serial
print(f"Serial execution time: {serial_time:.2f} seconds")

# Parallel execution
start_parallel = time.time()
results_parallel = parallel_chain.invoke({"topic": "Python programming"})
end_parallel = time.time()

parallel_time = end_parallel - start_parallel
print(f"Parallel execution time: {parallel_time:.2f} seconds")

print(f"\nSpeedup: {serial_time / parallel_time:.2f}x faster")

<div dir="rtl" align="right">

**نتیجه:**  
همان‌طور که می‌بینید، اجرای موازی معمولاً ۲ تا ۳ برابر سریع‌تر از اجرای سریالی است، چون درخواست‌ها همزمان ارسال می‌شوند.

</div>

<div dir="rtl" align="right">

---

### ۵. مثال پیشرفته: استفاده از ایجنت‌ها با ابزارهای مختلف به‌صورت موازی

در این بخش، یک ایجنت می‌سازیم که چند ابزار دارد و می‌تواند چند سؤال را همزمان پردازش کند.

**سناریو:**  
ما می‌خواهیم:
1. یک محاسبه ریاضی انجام دهیم
2. اطلاعات یک شهر را پیدا کنیم
3. یک توضیح کوتاه درباره یک موضوع بگیریم

</div>

In [ ]:
from langchain_core.tools import tool

@tool
def calculate(expression: str) -> str:
    """Evaluate a mathematical expression and return the result."""
    try:
        result = eval(expression)
        return f"The result is: {result}"
    except Exception as e:
        return f"Error: {str(e)}"

@tool
def get_city_info(city: str) -> str:
    """Get basic information about a city."""
    # Mock data for demonstration
    cities = {
        "Tehran": "Capital of Iran, population ~9 million",
        "Paris": "Capital of France, known for Eiffel Tower",
        "Tokyo": "Capital of Japan, largest metropolitan area"
    }
    return cities.get(city, f"No information available for {city}")

@tool
def explain_concept(concept: str) -> str:
    """Provide a brief explanation of a concept."""
    return f"Explaining {concept}: This is a placeholder explanation for demonstration purposes."

tools = [calculate, get_city_info, explain_concept]

In [ ]:
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate

# Create agent
agent_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that can use tools to answer questions."),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

agent = create_tool_calling_agent(llm, tools, agent_prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Test the agent with a single query first
test_result = agent_executor.invoke({"input": "Calculate 234 * 567"})
print("Test Result:", test_result["output"])

# Now run multiple queries in parallel
query1 = "Calculate 234 * 567"
query2 = "Tell me about Tehran"
query3 = "Explain quantum computing"

# Using RunnableParallel with agent
# Note: Each lambda needs to accept an input parameter even if not used
parallel_agent_chain = RunnableParallel(
    calculation=lambda _: agent_executor.invoke({"input": query1}),
    city_info=lambda _: agent_executor.invoke({"input": query2}),
    concept=lambda _: agent_executor.invoke({"input": query3})
)

results = parallel_agent_chain.invoke({})

print("\n" + "="*50)
print("Calculation Result:", results["calculation"]["output"])
print("\nCity Info:", results["city_info"]["output"])
print("\nConcept Explanation:", results["concept"]["output"])

<div dir="rtl" align="right">

**توضیح:**  
- سه ابزار مختلف ساختیم: محاسبه‌گر، اطلاعات شهر، و توضیح‌دهنده مفهوم
- یک ایجنت با این ابزارها ساختیم
- سه سؤال مختلف را به‌صورت موازی اجرا کردیم
- هر کدام از این سؤال‌ها می‌تواند از ابزار مناسب خود استفاده کند

**مزیت:** در یک سیستم واقعی، این روش می‌تواند زمان پاسخ‌دهی را به‌طور قابل‌توجهی کاهش دهد.

</div>

<div dir="rtl" align="right">

**نکات مهم درباره نسخه جدید LangChain:**

1. **تغییر import ابزارها:** در نسخه‌های جدید، `@tool` از `langchain_core.tools` import می‌شود نه `langchain.agents`
2. **lambda functions:** هنگام استفاده از `RunnableParallel` با agent executor، باید lambda ها یک پارامتر بگیرند (حتی اگر استفاده نشود)
3. **سازگاری:** کد بالا با LangChain 0.3.x و بالاتر سازگار است

</div>

<div dir="rtl" align="right">

---

### ۶. تمرین و جمع‌بندی

**تمرین‌های پیشنهادی:**

1. **تمرین ساده:** یک `RunnableParallel` بسازید که سه سؤال مختلف درباره یک زبان برنامه‌نویسی بپرسد (تاریخچه، مزایا، کاربردها).

2. **تمرین متوسط:** مقایسه زمان اجرا را برای ۵ سؤال مختلف انجام دهید و ببینید چقدر سرعت افزایش می‌یابد.

3. **تمرین پیشرفته:** یک سیستم بسازید که اطلاعات چند شهر را به‌صورت موازی دریافت کند و مقایسه کند.

4. **چالش:** ایجنتی بسازید که بتواند چند درخواست API را به‌صورت موازی اجرا کند و نتایج را ترکیب کند.

</div>

<div dir="rtl" align="right">

**جمع‌بندی نکات کلیدی:**

1. **RunnableParallel** ابزاری قدرتمند برای اجرای همزمان چند وظیفه است
2. اجرای موازی می‌تواند زمان پاسخ را ۲ تا ۳ برابر کاهش دهد
3. برای سیستم‌هایی که باید چند درخواست مستقل را پردازش کنند، این روش بسیار مفید است
4. می‌توان از این روش هم برای زنجیره‌های ساده و هم برای ایجنت‌ها استفاده کرد
5. در محیط‌های production، حتماً محدودیت‌های API rate limit را در نظر بگیرید

**منابع بیشتر:**
- [LangChain LCEL Documentation](https://python.langchain.com/docs/expression_language/)
- [RunnableParallel Guide](https://python.langchain.com/docs/expression_language/primitives/parallel)
- [Agent Documentation](https://python.langchain.com/docs/modules/agents/)

</div>